In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('new_sms_train.csv')
test_df = pd.read_csv('new_sms_test.csv')


In [2]:
train_df.head()

,text,label
0,5 Reasons You Need to Join InstaSpam Now!\n\n1...,spam
1,Win A FREE iPhone X Now!! Click Here To Enter!...,spam
2,CHECK OUT THESE AMAZING DEALS! GET RICH QUICK ...,spam
3,Urgent update!!1!!1!1 OMG 😱😱😱\nYou won't belie...,spam
4,Get rich quick with our amazing new investment...,spam


In [3]:
test_df.head()

,text,label
0,LeTs tAlK AbouT DisCOuNtS aNd DeAlS! WhO HaS T...,spam
1,Waaassssupppppp!!!!!! Have you checked out the...,spam
2,$$$MAKE $$$MONEY$$$ FAST!!!$$$ No scams!$$$ \n...,spam
3,Get rich quick with our amazing new product!!!...,spam
4,Are you tired of being single? Join our websit...,spam


In [4]:
len(train_df)

9950

In [5]:
len(test_df)

950

In [8]:
# Define the mapping
label_mapping = {"spam": 0, "not_spam": 1}

# Apply the mapping to the 'label' column
train_df['label'] = train_df['label'].map(label_mapping)
test_df['label'] = test_df['label'].map(label_mapping)


In [9]:
train_df.head()

,text,label
0,5 Reasons You Need to Join InstaSpam Now!\n\n1...,0
1,Win A FREE iPhone X Now!! Click Here To Enter!...,0
2,CHECK OUT THESE AMAZING DEALS! GET RICH QUICK ...,0
3,Urgent update!!1!!1!1 OMG 😱😱😱\nYou won't belie...,0
4,Get rich quick with our amazing new investment...,0


In [10]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from spellchecker import SpellChecker
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re

In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model

# Assuming train_df and test_df are your training and testing DataFrames respectively

# Tokenization for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Encoding as numbers for training dataset with tokenization
encoded_train_texts = tokenizer(train_df['text'].tolist(), padding=True, truncation=True, max_length=128, return_tensors='tf')
X_train_padded = pad_sequences(encoded_train_texts['input_ids'], padding='post', maxlen=128)
y_train = train_df['label'].to_numpy()

# Encoding as numbers for testing dataset with tokenization
encoded_test_texts = tokenizer(test_df['text'].tolist(), padding=True, truncation=True, max_length=128, return_tensors='tf')
X_test_padded = pad_sequences(encoded_test_texts['input_ids'], padding='post', maxlen=128)
y_test = test_df['label'].to_numpy()

# Building the custom BERT + DNN model
def create_bert_dnn_model():
    # Load the pre-trained BERT model
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')
    
    # Input layers
    input_ids = Input(shape=(128,), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(128,), dtype=tf.int32, name='attention_mask')
    
    # BERT outputs
    bert_outputs = bert_model(input_ids, attention_mask=attention_mask)
    sequence_output = bert_outputs.last_hidden_state
    pooled_output = bert_outputs.pooler_output
    
    # Custom DNN layers
    dropout = Dropout(0.3)(pooled_output)
    dense1 = Dense(256, activation='relu')(dropout)
    dropout1 = Dropout(0.3)(dense1)
    dense2 = Dense(64, activation='relu')(dropout1)
    dropout2 = Dropout(0.3)(dense2)
    output = Dense(2, activation='softmax')(dropout2)
    
    # Define the model
    model = Model(inputs=[input_ids, attention_mask], outputs=output)
    
    return model

# Create the model
model = create_bert_dnn_model()

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training the model with early stopping for 1 epoch
history = model.fit(
    {'input_ids': X_train_padded, 'attention_mask': encoded_train_texts['attention_mask']},
    y_train,
    epochs=1,
    batch_size=64,
    validation_split=0.3,
    callbacks=[early_stopping]
)

# Evaluating the model on the testing dataset
eval_result = model.evaluate(
    {'input_ids': X_test_padded, 'attention_mask': encoded_test_texts['attention_mask']},
    y_test
)
print(f"Test accuracy: {eval_result[1]}")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

30/30 [==============================] - 95s 3s/step - loss: 0.0141 - accuracy: 0.9958
Test accuracy: 0.9957894682884216


In [13]:
from sklearn.metrics import classification_report

# Predicting on the test set
y_pred = model.predict({'input_ids': X_test_padded, 'attention_mask': encoded_test_texts['attention_mask']})
y_pred = np.argmax(y_pred, axis=1)

# Generating the classification report without rounding off
report = classification_report(y_test, y_pred, digits=4)
print(report)


30/30 [==============================] - 94s 3s/step
              precision    recall  f1-score   support

           0     0.9940    0.9980    0.9960       500
           1     0.9978    0.9933    0.9955       450

    accuracy                         0.9958       950
   macro avg     0.9959    0.9957    0.9958       950
weighted avg     0.9958    0.9958    0.9958       950

